In [2]:
import soccerdata as sd
import warnings
warnings.filterwarnings(module='soccerdata', action='ignore')

In [ ]:
fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons = '24-25')

In [5]:
stat_list = ['standard', 'shooting', 'passing', 'passing_types', 'goal_shot_creation', 'defense', 'possession', 'playing_time', 'misc', 'keeper', 'keeper_adv']

In [ ]:
for stat in stat_list:
    try:
        player_season_stats = fbref.read_player_season_stats(stat_type=stat)
        #print(player_season_stats.columns)
    except Exception as e:
        print(f"Error processing {stat}: {e}")

In [66]:
cont_NO_GK = []
only_GK = []
for stat in stat_list:
    try:
        player_season_stats = fbref.read_player_season_stats(stat_type=stat)
        positions = player_season_stats['pos'].unique().tolist()
        if 'GK' not in positions:
            cont_NO_GK.append(stat)
        elif len(positions) == 1:
            only_GK.append(stat)
        #print(player_season_stats.columns)
    except Exception as e:
        print(f"Error processing {stat}: {e}")
        
print("Stats without Goalkeepers:", cont_NO_GK)
print("Stats with only Goalkeepers:", only_GK)

Stats without Goalkeepers: []
Stats with only Goalkeepers: ['keeper', 'keeper_adv']


In [4]:
player_season_stats_standard = fbref.read_player_season_stats(stat_type='standard')
player_season_stats_shooting = fbref.read_player_season_stats(stat_type='shooting')
player_season_stats_keeper = fbref.read_player_season_stats(stat_type='keeper')

In [5]:
print("Unique positions in Standard Stats:", player_season_stats_standard['pos'].unique().tolist())

Unique positions in Standard Stats: ['DF', 'FW,MF', 'GK', 'MF', 'FW', 'DF,FW', 'MF,FW', 'DF,MF', 'MF,DF', 'FW,DF']


In [13]:
# Check if dataframes share a common column for merging
common_columns = set(player_season_stats_standard.columns).intersection(set(player_season_stats_shooting.columns))
print("Common columns to avoid at merging:", common_columns)

Common columns to avoid at merging: {('nation', ''), ('born', ''), ('Expected', 'npxG'), ('age', ''), ('Expected', 'xG'), ('pos', '')}


In [ ]:
distinct_columns = player_season_stats_shooting.columns.difference(player_season_stats_standard.columns)
print("Distinct columns in shooting stats:", distinct_columns.tolist())
player_season_combined = player_season_stats_standard.join(player_season_stats_shooting[distinct_columns], how='left', rsuffix='_drop')

# Check if dataframe has any columns with '_drop' suffix
drop_columns = [col for col in player_season_combined.columns if col[0].endswith('_drop')]
print("Columns with '_drop' suffix after merging:", drop_columns)
print(" Columns in combined dataframe:", player_season_combined.columns.to_list())
columns_name_combined = [' '.join(col[::-1]).strip(' ') for col in player_season_combined.columns]
print("Combined columns names:", columns_name_combined)


Distinct columns in shooting stats: [('90s', ''), ('Expected', 'G-xG'), ('Expected', 'np:G-xG'), ('Expected', 'npxG/Sh'), ('Standard', 'Dist'), ('Standard', 'FK'), ('Standard', 'G/Sh'), ('Standard', 'G/SoT'), ('Standard', 'Gls'), ('Standard', 'PK'), ('Standard', 'PKatt'), ('Standard', 'Sh'), ('Standard', 'Sh/90'), ('Standard', 'SoT'), ('Standard', 'SoT%'), ('Standard', 'SoT/90')]
Columns with '_drop' suffix after merging: []
 Columns in combined dataframe: [('nation', ''), ('pos', ''), ('age', ''), ('born', ''), ('Playing Time', 'MP'), ('Playing Time', 'Starts'), ('Playing Time', 'Min'), ('Playing Time', '90s'), ('Performance', 'Gls'), ('Performance', 'Ast'), ('Performance', 'G+A'), ('Performance', 'G-PK'), ('Performance', 'PK'), ('Performance', 'PKatt'), ('Performance', 'CrdY'), ('Performance', 'CrdR'), ('Expected', 'xG'), ('Expected', 'npxG'), ('Expected', 'xAG'), ('Expected', 'npxG+xAG'), ('Progression', 'PrgC'), ('Progression', 'PrgP'), ('Progression', 'PrgR'), ('Per 90 Minutes', '

In [38]:
player_season_combined_ = player_season_combined.copy()
player_season_combined_.columns = [' '.join(col[::-1]).strip(' ') for col in player_season_combined_.columns]
print("Combined dataframe with flattened columns:", player_season_combined_.columns.to_list())
player_season_combined_.reset_index(inplace=True)

player_season_combined_.head(4)

Combined dataframe with flattened columns: ['nation', 'pos', 'age', 'born', 'MP Playing Time', 'Starts Playing Time', 'Min Playing Time', '90s Playing Time', 'Gls Performance', 'Ast Performance', 'G+A Performance', 'G-PK Performance', 'PK Performance', 'PKatt Performance', 'CrdY Performance', 'CrdR Performance', 'xG Expected', 'npxG Expected', 'xAG Expected', 'npxG+xAG Expected', 'PrgC Progression', 'PrgP Progression', 'PrgR Progression', 'Gls Per 90 Minutes', 'Ast Per 90 Minutes', 'G+A Per 90 Minutes', 'G-PK Per 90 Minutes', 'G+A-PK Per 90 Minutes', 'xG Per 90 Minutes', 'xAG Per 90 Minutes', 'xG+xAG Per 90 Minutes', 'npxG Per 90 Minutes', 'npxG+xAG Per 90 Minutes', '90s', 'G-xG Expected', 'np:G-xG Expected', 'npxG/Sh Expected', 'Dist Standard', 'FK Standard', 'G/Sh Standard', 'G/SoT Standard', 'Gls Standard', 'PK Standard', 'PKatt Standard', 'Sh Standard', 'Sh/90 Standard', 'SoT Standard', 'SoT% Standard', 'SoT/90 Standard']


,league,season,team,player,nation,pos,age,born,MP Playing Time,Starts Playing Time,...,G/Sh Standard,G/SoT Standard,Gls Standard,PK Standard,PKatt Standard,Sh Standard,Sh/90 Standard,SoT Standard,SoT% Standard,SoT/90 Standard
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26,1997,17,13,...,0.0,0.0,0,0,0,9,0.68,2,22.2,0.15
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22,2001,25,20,...,0.08,0.23,6,1,1,66,3.44,22,33.3,1.15
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28,1995,38,38,...,<NA>,<NA>,0,0,0,0,0.0,0,<NA>,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25,1999,35,33,...,0.08,0.29,4,0,0,48,1.53,14,29.2,0.45


In [10]:
keeper_combined = player_season_stats_keeper.join(player_season_stats_standard, how='left', rsuffix='_standard')
print(keeper_combined.columns.to_list())

[('nation', ''), ('pos', ''), ('age', ''), ('born', ''), ('Playing Time', 'MP'), ('Playing Time', 'Starts'), ('Playing Time', 'Min'), ('90s', ''), ('Performance', 'GA'), ('Performance', 'GA90'), ('Performance', 'SoTA'), ('Performance', 'Saves'), ('Performance', 'Save%'), ('Performance', 'W'), ('Performance', 'D'), ('Performance', 'L'), ('Performance', 'CS'), ('Performance', 'CS%'), ('Penalty Kicks', 'PKatt'), ('Penalty Kicks', 'PKA'), ('Penalty Kicks', 'PKsv'), ('Penalty Kicks', 'PKm'), ('Penalty Kicks', 'Save%'), ('nation_standard', ''), ('pos_standard', ''), ('age_standard', ''), ('born_standard', ''), ('Playing Time_standard', 'MP'), ('Playing Time_standard', 'Starts'), ('Playing Time_standard', 'Min'), ('Playing Time_standard', '90s'), ('Performance', 'Gls'), ('Performance', 'Ast'), ('Performance', 'G+A'), ('Performance', 'G-PK'), ('Performance', 'PK'), ('Performance', 'PKatt'), ('Performance', 'CrdY'), ('Performance', 'CrdR'), ('Expected', 'xG'), ('Expected', 'npxG'), ('Expected',

In [82]:
# Join keeper stats with standard stats on their common index
keeper_combined = player_season_stats_keeper.join(player_season_stats_standard, rsuffix='_standard')
keeper_combined.head()

nation pos age  born  \
                                                                               
league             season team        player                                   
ENG-Premier League 2425   Arsenal     David Raya           ESP  GK  28  1995   
                          Aston Villa Emiliano Martínez    ARG  GK  31  1992   
                                      Robin Olsen          SWE  GK  34  1990   
                          Bournemouth Kepa Arrizabalaga    ESP  GK  29  1994   
                                      Mark Travers         IRL  GK  25  1999   

                                                        Playing Time         \
                                                                  MP Starts   
league             season team        player                                  
ENG-Premier League 2425   Arsenal     David Raya                  38     38   
                          Aston Villa Emiliano Martínez           37     37   
                                      Robin Olsen                  4      1   
                          Bournemouth Kepa Arrizabalaga           31     31   
                                      Mark Travers                 5      5   

                                                                90s  \
                                                          Min         
league             season team        player                          
ENG-Premier League 2425   Arsenal     David Raya         3420  38.0   
                          Aston Villa Emiliano Martínez  3194  35.5   
                                      Robin Olsen         226   2.5   
                          Bournemouth Kepa Arrizabalaga  2790  31.0   
                                      Mark Travers        450   5.0   

                                                        Performance        \
                                                                 GA  GA90   
league             season team        player                                
ENG-Premier League 2425   Arsenal     David Raya                 34  0.89   
                          Aston Villa Emiliano Martínez          45  1.27   
                                      Robin Olsen                 6  2.39   
                          Bournemouth Kepa Arrizabalaga          39  1.26   
                                      Mark Travers                5   1.0   

                                                         ... Per 90 Minutes  \
                                                         ...            Gls   
league             season team        player             ...                  
ENG-Premier League 2425   Arsenal     David Raya         ...            0.0   
                          Aston Villa Emiliano Martínez  ...            0.0   
                                      Robin Olsen        ...            0.0   
                          Bournemouth Kepa Arrizabalaga  ...            0.0   
                                      Mark Travers       ...            0.0   

                                                                               \
                                                         Ast  G+A G-PK G+A-PK   
league             season team        player                                    
ENG-Premier League 2425   Arsenal     David Raya         0.0  0.0  0.0    0.0   
                          Aston Villa Emiliano Martínez  0.0  0.0  0.0    0.0   
                                      Robin Olsen        0.0  0.0  0.0    0.0   
                          Bournemouth Kepa Arrizabalaga  0.0  0.0  0.0    0.0   
                                      Mark Travers       0.0  0.0  0.0    0.0   

                                                                           \
                                                          xG   xAG xG+xAG   
league             season team        player                                
ENG-Premier League 2425   Arsenal     David Raya         0.0   0.0    0.0   
      

In [80]:
combined = player_season_stats_standard.join(player_season_stats_shooting, rsuffix='_shooting')
combined.head()

nation    pos age  born  \
                                                                          
league             season team    player                                  
ENG-Premier League 2425   Arsenal Ben White        ENG     DF  26  1997   
                                  Bukayo Saka      ENG  FW,MF  22  2001   
                                  David Raya       ESP     GK  28  1995   
                                  Declan Rice      ENG     MF  25  1999   
                                  Ethan Nwaneri    ENG  FW,MF  17  2007   

                                                Playing Time               \
                                                          MP Starts   Min   
league             season team    player                                    
ENG-Premier League 2425   Arsenal Ben White               17     13  1198   
                                  Bukayo Saka             25     20  1729   
                                  David Raya              38     38  3420   
                                  Declan Rice             35     33  2825   
                                  Ethan Nwaneri           26     11   895   

                                                      Performance      ...  \
                                                  90s         Gls Ast  ...   
league             season team    player                               ...   
ENG-Premier League 2425   Arsenal Ben White      13.3           0   2  ...   
                                  Bukayo Saka    19.2           6  10  ...   
                                  David Raya     38.0           0   0  ...   
                                  Declan Rice    31.4           4   7  ...   
                                  Ethan Nwaneri   9.9           4   2  ...   

                                                Standard                    \
                                                   G/SoT  Dist FK PK PKatt   
league             season team    player                                     
ENG-Premier League 2425   Arsenal Ben White          0.0  19.4  0  0     0   
                                  Bukayo Saka       0.23  15.6  1  1     1   
                                  David Raya        <NA>  <NA>  0  0     0   
                                  Declan Rice       0.29  19.6  2  0     0   
                                  Ethan Nwaneri     0.44  18.5  1  0     0   

                                                Expected_shooting       \
                                                               xG npxG   
league             season team    player                                 
ENG-Premier League 2425   Arsenal Ben White                   0.5  0.5   
                                  Bukayo Saka                 6.8  6.0   
                                  David Raya                  0.0  0.0   
                                  Declan Rice                 3.5  3.5   
                                  Ethan Nwaneri               1.2  1.2   

                                                                      
                                                npxG/Sh G-xG np:G-xG  
league             season team    player                              
ENG-Premier League 2425   Arsenal Ben White        0.05 -0.5    -0.5  
                                  Bukayo Saka      0.09 -0.8    -1.0  
                                  David Raya       <NA>  0.0     0.0  
                                  Declan Rice      0.07  0.5     0.5  
                                  Ethan Nwaneri    0.05  2.8     2.8  

[5 rows x 55 columns]

In [68]:
columns_name_combined = ['_'.join(col).strip('_') for col in combined.columns]
print(columns_name_combined)

['nation', 'pos', 'age', 'born', 'Playing Time_MP', 'Playing Time_Starts', 'Playing Time_Min', 'Playing Time_90s', 'Performance_Gls', 'Performance_Ast', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR', 'Expected_xG', 'Expected_npxG', 'Expected_xAG', 'Expected_npxG+xAG', 'Progression_PrgC', 'Progression_PrgP', 'Progression_PrgR', 'Per 90 Minutes_Gls', 'Per 90 Minutes_Ast', 'Per 90 Minutes_G+A', 'Per 90 Minutes_G-PK', 'Per 90 Minutes_G+A-PK', 'Per 90 Minutes_xG', 'Per 90 Minutes_xAG', 'Per 90 Minutes_xG+xAG', 'Per 90 Minutes_npxG', 'Per 90 Minutes_npxG+xAG', 'nation_shooting', 'pos_shooting', 'age_shooting', 'born_shooting', '90s', 'Standard_Gls', 'Standard_Sh', 'Standard_SoT', 'Standard_SoT%', 'Standard_Sh/90', 'Standard_SoT/90', 'Standard_G/Sh', 'Standard_G/SoT', 'Standard_Dist', 'Standard_FK', 'Standard_PK', 'Standard_PKatt', 'Expected_shooting_xG', 'Expected_shooting_npxG', 'Expected_shooting_npxG/Sh', 'Expected_sho

In [48]:
columns_name_list = ['_'.join(col).strip('_') for col in player_season_stats.columns]
print(columns_name_list)

['nation', 'pos', 'age', 'born', '90s', 'Goals_GA', 'Goals_PKA', 'Goals_FK', 'Goals_CK', 'Goals_OG', 'Expected_PSxG', 'Expected_PSxG/SoT', 'Expected_PSxG+/-', 'Expected_/90', 'Launched_Cmp', 'Launched_Att', 'Launched_Cmp%', 'Passes_Att (GK)', 'Passes_Thr', 'Passes_Launch%', 'Passes_AvgLen', 'Goal Kicks_Att', 'Goal Kicks_Launch%', 'Goal Kicks_AvgLen', 'Crosses_Opp', 'Crosses_Stp', 'Crosses_Stp%', 'Sweeper_#OPA', 'Sweeper_#OPA/90', 'Sweeper_AvgDist']


In [81]:
combined.columns = ['_'.join(col).strip('_') for col in combined.columns]
combined.reset_index(inplace=True)
combined.head()

,league,season,team,player,nation,pos,age,born,Playing Time_MP,Playing Time_Starts,...,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_shooting_xG,Expected_shooting_npxG,Expected_shooting_npxG/Sh,Expected_shooting_G-xG,Expected_shooting_np:G-xG
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26,1997,17,13,...,0.0,19.4,0,0,0,0.5,0.5,0.05,-0.5,-0.5
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22,2001,25,20,...,0.23,15.6,1,1,1,6.8,6.0,0.09,-0.8,-1.0
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28,1995,38,38,...,<NA>,<NA>,0,0,0,0.0,0.0,<NA>,0.0,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25,1999,35,33,...,0.29,19.6,2,0,0,3.5,3.5,0.07,0.5,0.5
4,ENG-Premier League,2425,Arsenal,Ethan Nwaneri,ENG,"FW,MF",17,2007,26,11,...,0.44,18.5,1,0,0,1.2,1.2,0.05,2.8,2.8


In [51]:
player_season_stats.columns.values

array(['league', 'season', 'team', 'player', 'nation', 'pos', 'age',
       'born', '90s', 'Goals_GA', 'Goals_PKA', 'Goals_FK', 'Goals_CK',
       'Goals_OG', 'Expected_PSxG', 'Expected_PSxG/SoT',
       'Expected_PSxG+/-', 'Expected_/90', 'Launched_Cmp', 'Launched_Att',
       'Launched_Cmp%', 'Passes_Att (GK)', 'Passes_Thr', 'Passes_Launch%',
       'Passes_AvgLen', 'Goal Kicks_Att', 'Goal Kicks_Launch%',
       'Goal Kicks_AvgLen', 'Crosses_Opp', 'Crosses_Stp', 'Crosses_Stp%',
       'Sweeper_#OPA', 'Sweeper_#OPA/90', 'Sweeper_AvgDist'], dtype=object)

In [ ]:
cols_level_0 = player_season_stats.columns.droplevel([1]).to_list()[:4]
cols_level_0

['nation', 'pos', 'age', 'born']

In [5]:
player_season_stats.reset_index(inplace=True,col_level=1)
player_season_stats.head()

nation    pos age  born  \
               league season     team         player                           
0  ENG-Premier League   2425  Arsenal      Ben White    ENG     DF  26  1997   
1  ENG-Premier League   2425  Arsenal    Bukayo Saka    ENG  FW,MF  22  2001   
2  ENG-Premier League   2425  Arsenal     David Raya    ESP     GK  28  1995   
3  ENG-Premier League   2425  Arsenal    Declan Rice    ENG     MF  25  1999   
4  ENG-Premier League   2425  Arsenal  Ethan Nwaneri    ENG  FW,MF  17  2007   

  Playing Time         ... Per 90 Minutes                                 \
            MP Starts  ...            Gls   Ast   G+A  G-PK G+A-PK    xG   
0           17     13  ...            0.0  0.15  0.15   0.0   0.15  0.03   
1           25     20  ...           0.31  0.52  0.83  0.26   0.78  0.36   
2           38     38  ...            0.0   0.0   0.0   0.0    0.0   0.0   
3           35     33  ...           0.13  0.22  0.35  0.13   0.35  0.11   
4           26     11  ...            0.4   0.2   0.6   0.4    0.6  0.12   

                               
    xAG xG+xAG  npxG npxG+xAG  
0   0.1   0.14  0.03     0.14  
1   0.4   0.75  0.31     0.71  
2   0.0    0.0   0.0      0.0  
3  0.21   0.32  0.11     0.32  
4  0.12   0.24  0.12     0.24  

[5 rows x 37 columns]

In [6]:
cols_level_1 = player_season_stats.columns.droplevel([0]).values.tolist()

columns_90min = [f"{col} per 90min" for col in cols_level_1[-10:]]

column_names = cols_level_1[:4]+cols_level_0+cols_level_1[8:-10]+columns_90min
player_season_stats.columns = column_names
player_season_stats.head()

,league,season,team,player,nation,pos,age,born,MP,Starts,...,Gls per 90min,Ast per 90min,G+A per 90min,G-PK per 90min,G+A-PK per 90min,xG per 90min,xAG per 90min,xG+xAG per 90min,npxG per 90min,npxG+xAG per 90min
0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26,1997,17,13,...,0.0,0.15,0.15,0.0,0.15,0.03,0.1,0.14,0.03,0.14
1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22,2001,25,20,...,0.31,0.52,0.83,0.26,0.78,0.36,0.4,0.75,0.31,0.71
2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28,1995,38,38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25,1999,35,33,...,0.13,0.22,0.35,0.13,0.35,0.11,0.21,0.32,0.11,0.32
4,ENG-Premier League,2425,Arsenal,Ethan Nwaneri,ENG,"FW,MF",17,2007,26,11,...,0.4,0.2,0.6,0.4,0.6,0.12,0.12,0.24,0.12,0.24


In [7]:
player_season_stats.to_csv("big5_player_season_stats_24_25.csv", index=False)

In [8]:
team_stats = fbref.read_team_season_stats(stat_type="standard", opponent_stats=False)
team_stats.head()

[11/11/25 18:53:53] WARNING  /home/edgar25/miniconda3/envs/CompTDSc/lib/python3.12/site-packages/so ]8;id=196875;file:///home/edgar25/miniconda3/envs/CompTDSc/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=41021;file:///home/edgar25/miniconda3/envs/CompTDSc/lib/python3.12/warnings.py#112\112]8;;\
                             ccerdata/fbref.py:165: FutureWarning: The behavior of DataFrame                       
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

players_used   Age  Poss Playing Time  \
                                                                         MP   
league             season team                                                
ENG-Premier League 2425   Arsenal               25  25.8  56.9           38   
                          Aston Villa           28  27.0  50.5           38   
                          Bournemouth           29  25.1  48.5           38   
                          Brentford             28  25.8  47.9           38   
                          Brighton              32  24.8  52.3           38   

                                                       Performance           \
                                      Starts   Min 90s         Gls Ast  G+A   
league             season team                                                
ENG-Premier League 2425   Arsenal        418  3420  38          67  55  122   
                          Aston Villa    418  3420  38          56  45  101   
                          Bournemouth    418  3420  38          57  41   98   
                          Brentford      418  3420  38          65  44  109   
                          Brighton       418  3420  38          64  41  105   

                                       ... Per 90 Minutes                     \
                                       ...            Ast   G+A  G-PK G+A-PK   
league             season team         ...                                     
ENG-Premier League 2425   Arsenal      ...           1.45  3.21  1.71   3.16   
                          Aston Villa  ...           1.18  2.66  1.39   2.58   
                          Bournemouth  ...           1.08  2.58  1.34   2.42   
                          Brentford    ...           1.16  2.87  1.58   2.74   
                          Brighton     ...           1.08  2.76   1.5   2.58   

                                                                         \
                                         xG   xAG xG+xAG  npxG npxG+xAG   
league             season team                                            
ENG-Premier League 2425   Arsenal      1.58  1.19   2.77  1.54     2.73   
                          Aston Villa  1.48   1.1   2.57  1.35     2.45   
                          Bournemouth  1.68  1.15   2.84  1.54     2.69   
                          Brentford    1.55  1.12   2.67  1.43     2.55   
                          Brighton     1.54  1.06   2.61   1.4     2.47   

                                                                                     url  
                                                                                          
league             season team                                                            
ENG-Premier League 2425   Arsenal            /en/squads/18bb7c10/2024-2025/Arsenal-Stats  
                          Aston Villa    /en/squads/8602292d/2024-2025/Aston-Villa-Stats  
                          Bournemouth    /en/squads/4ba7cbea/2024-2025/Bournemouth-Stats  
                          Brentford        /en/squads/cd051869/2024-2025/Brentford-Stats  
                          Brighton     /en/squads/d07537b9/2024-2025/Brighton-and-Hov...  

[5 rows x 32 columns]